## Does nn.Conv2d init work well?

In [0]:
!curl https://course.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   322  100   322    0     0   1610      0 --:--:-- --:--:-- --:--:--  1610
Updating fastai...
Done.


[Jump_to lesson 9 video](https://course.fast.ai/videos/?lesson=9&t=21)

In [0]:
#export
from exp.nb_02 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

ModuleNotFoundError: ignored

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import torch

In [0]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [0]:
torch.nn
def reset_parameters(self):
    init.kaiming_uniform_(self.weight, a=math.sqrt(5))
    if self.bias is not None:
        fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
        bound = 1 / math.sqrt(fan_in)
        init.uniform_(self.bias, -bound, bound)

In [0]:
from torch.nn import init

In [0]:
init._calculate_fan_in_and_fan_out??

In [0]:
def _calculate_fan_in_and_fan_out(tensor):
    dimensions = tensor.dim()
    if dimensions < 2:
        raise ValueError("Fan in and fan out can not be computed for tensor with fewer than 2 dimensions")

    if dimensions == 2:  # Linear
        fan_in = tensor.size(1)
        fan_out = tensor.size(0)
    else:
        num_input_fmaps = tensor.size(1)
        num_output_fmaps = tensor.size(0)
        receptive_field_size = 1
        if tensor.dim() > 2:
            receptive_field_size = tensor[0][0].numel()
        fan_in = num_input_fmaps * receptive_field_size
        fan_out = num_output_fmaps * receptive_field_size

    return fan_in, fan_out

In [0]:
x_train,y_train,x_valid,y_valid = get_data()
train_mean,train_std = x_train.mean(),x_train.std()
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [0]:
x_train = x_train.view(-1,1,28,28)
x_valid = x_valid.view(-1,1,28,28)
x_train.shape,x_valid.shape

(torch.Size([50000, 1, 28, 28]), torch.Size([10000, 1, 28, 28]))

In [0]:
n,*_ = x_train.shape
c = y_train.max()+1
nh = 32
n,c

(50000, tensor(10))

In [0]:
l1 = nn.Conv2d(1, nh, 5)

In [0]:
x = x_valid[:100]

In [0]:
x.shape

torch.Size([100, 1, 28, 28])

In [0]:
def stats(x): return x.mean(),x.std()

In [0]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [0]:
stats(l1.weight),stats(l1.bias)

((tensor(-0.0043, grad_fn=<MeanBackward1>),
  tensor(0.1156, grad_fn=<StdBackward0>)),
 (tensor(0.0212, grad_fn=<MeanBackward1>),
  tensor(0.1176, grad_fn=<StdBackward0>)))

In [0]:
t = l1(x)

In [0]:
stats(t)

(tensor(0.0107, grad_fn=<MeanBackward1>),
 tensor(0.5978, grad_fn=<StdBackward0>))

In [0]:
init.kaiming_normal_(l1.weight, a=1.)
stats(l1(x))

(tensor(0.0267, grad_fn=<MeanBackward1>),
 tensor(1.1067, grad_fn=<StdBackward0>))

In [0]:
import torch.nn.functional as F

In [0]:
def f1(x,a=0): return F.leaky_relu(l1(x),a)

In [0]:
init.kaiming_normal_(l1.weight, a=0)
stats(f1(x))

(tensor(0.5547, grad_fn=<MeanBackward1>),
 tensor(1.0199, grad_fn=<StdBackward0>))

In [0]:
l1 = nn.Conv2d(1, nh, 5)
stats(f1(x))

(tensor(0.2219, grad_fn=<MeanBackward1>),
 tensor(0.3653, grad_fn=<StdBackward0>))

In [0]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [0]:
# receptive field size
rec_fs = l1.weight[0,0].numel()
rec_fs

25

In [0]:
nf,ni,*_ = l1.weight.shape
nf,ni

(32, 1)

In [0]:
fan_in  = ni*rec_fs
fan_out = nf*rec_fs
fan_in,fan_out

(25, 800)

In [0]:
def gain(a): return math.sqrt(2.0 / (1 + a**2))

In [0]:
gain(1),gain(0),gain(0.01),gain(0.1),gain(math.sqrt(5.))

(1.0,
 1.4142135623730951,
 1.4141428569978354,
 1.4071950894605838,
 0.5773502691896257)

In [0]:
torch.zeros(10000).uniform_(-1,1).std()

tensor(0.5788)

In [0]:
1/math.sqrt(3.)

0.5773502691896258

In [0]:
def kaiming2(x,a, use_fan_out=False):
    nf,ni,*_ = x.shape
    rec_fs = x[0,0].shape.numel()
    fan = nf*rec_fs if use_fan_out else ni*rec_fs
    std = gain(a) / math.sqrt(fan)
    bound = math.sqrt(3.) * std
    x.data.uniform_(-bound,bound)

In [0]:
kaiming2(l1.weight, a=0);
stats(f1(x))

(tensor(0.5603, grad_fn=<MeanBackward1>),
 tensor(1.0921, grad_fn=<StdBackward0>))

In [0]:
kaiming2(l1.weight, a=math.sqrt(5.))
stats(f1(x))

(tensor(0.2186, grad_fn=<MeanBackward1>),
 tensor(0.3437, grad_fn=<StdBackward0>))

In [0]:
class Flatten(nn.Module):
    def forward(self,x): return x.view(-1)

In [0]:
m = nn.Sequential(
    nn.Conv2d(1,8, 5,stride=2,padding=2), nn.ReLU(),
    nn.Conv2d(8,16,3,stride=2,padding=1), nn.ReLU(),
    nn.Conv2d(16,32,3,stride=2,padding=1), nn.ReLU(),
    nn.Conv2d(32,1,3,stride=2,padding=1),
    nn.AdaptiveAvgPool2d(1),
    Flatten(),
)

In [0]:
y = y_valid[:100].float()

In [0]:
t = m(x)
stats(t)

(tensor(0.0875, grad_fn=<MeanBackward1>),
 tensor(0.0065, grad_fn=<StdBackward0>))

In [0]:
l = mse(t,y)
l.backward()

In [0]:
stats(m[0].weight.grad)

(tensor(0.0054), tensor(0.0333))

In [0]:
init.kaiming_uniform_??

In [0]:
for l in m:
    if isinstance(l,nn.Conv2d):
        init.kaiming_uniform_(l.weight)
        l.bias.data.zero_()

In [0]:
t = m(x)
stats(t)

(tensor(-0.0352, grad_fn=<MeanBackward1>),
 tensor(0.4043, grad_fn=<StdBackward0>))

In [0]:
l = mse(t,y)
l.backward()
stats(m[0].weight.grad)

(tensor(0.0093), tensor(0.4231))

## Export

In [0]:
!./notebook2script.py 02a_why_sqrt5.ipynb